<a href="https://colab.research.google.com/github/ghlim40/BT/blob/main/%ED%8C%8C%EC%9D%B4%EC%8D%AC%EC%9C%BC%EB%A1%9C_%ED%95%98%EB%8A%94_%EB%B0%B1%ED%85%8C%EC%8A%A4%ED%8C%85_%EA%B3%A8%EB%93%A0%ED%81%AC%EB%A1%9C%EC%8A%A4_%EA%B0%80%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
#출처 https://www.youtube.com/watch?v=XNjY7p_0Xm4
from pykrx import stock
import numpy as np 
import pandas as pd
from pandas import Series

In [6]:
def 장기_단기_이평선_수익률(df, Ns, Nl):
    df = df[  [ '종가' ] ].copy()
    df['ma_s'] = df['종가'].rolling( Ns ).mean( ).shift ( 1 )
    df['ma_l'] = df['종가'].rolling( Nl ).mean( ).shift ( 1 )
    cond = df['ma_s'] > df['ma_l']
    df['status'] = np.where(cond, 1, 0)
    df.iloc[-1, -1] = 0
    
    # 매수/매도 조건 
    매수조건 = (df['status'] == 1) & (df['status'].shift(1) != 1)
    매도조건 = (df['status'] == 0) & (df['status'].shift(1) == 1)
    
    # 수익률 계산
    수익률 = df.loc[매도조건, '종가'].reset_index(drop=True) / df.loc[매수조건, '종가'].reset_index(drop=True)
    수익률 = 수익률 - 0.002
    return 수익률.cumprod().iloc[-1]

In [11]:
df = stock.get_index_ohlcv_by_date("20200601", "20210201", "1001")
df = df[  [ '종가' ] ]

In [12]:
result = [ ]
for i in range(2, 17):
    for j in range(30, 61):
        result.append(장기_단기_이평선_수익률(df, i, j))

In [13]:
index = pd.MultiIndex.from_product([ range(2, 17), range(30, 61)  ])
s = Series(result, index)

In [14]:
print(s.idxmax())
print(s.max())

(14, 30)
1.3702115535912045
